In [3]:
import pandas as pd
import numpy as np

# # Load the dataset
netflix_data = pd.read_csv("C:/Users/DELL/Documents/Netflix Dataset/netflix_titles_nov_2019.csv")
print(netflix_data.head())



    show_id                                title                    director  \
0  81193313                            Chocolate                         NaN   
1  81197050  Guatemala: Heart of the Mayan World  Luis Ara, Ignacio Jaunsolo   
2  81213894                      The Zoya Factor             Abhishek Sharma   
3  81082007                            Atlantics                   Mati Diop   
4  80213643                      Chip and Potato                         NaN   

                                                cast  \
0  Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...   
1                                  Christian Morales   
2  Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...   
3  Mama Sane, Amadou Mbow, Ibrahima Traore, Nicol...   
4  Abigail Oliver, Andrea Libman, Briana Buckmast...   

                    country         date_added  release_year rating  \
0               South Korea  November 30, 2019          2019  TV-14   
1                       NaN  November 30

In [4]:
print(netflix_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       5837 non-null   int64 
 1   title         5837 non-null   object
 2   director      3936 non-null   object
 3   cast          5281 non-null   object
 4   country       5410 non-null   object
 5   date_added    5195 non-null   object
 6   release_year  5837 non-null   int64 
 7   rating        5827 non-null   object
 8   duration      5837 non-null   object
 9   listed_in     5837 non-null   object
 10  description   5837 non-null   object
 11  type          5837 non-null   object
dtypes: int64(2), object(10)
memory usage: 547.3+ KB
None


In [5]:
# Checking null values
print(netflix_data.isnull().sum())

show_id            0
title              0
director        1901
cast             556
country          427
date_added       642
release_year       0
rating            10
duration           0
listed_in          0
description        0
type               0
dtype: int64


In [6]:
print("Total duplicate rows:", netflix_data.duplicated().sum())

Total duplicate rows: 0


In [7]:
#Fill null values with Not Rated in rating column
netflix_data['rating'] = netflix_data['rating'].fillna('Not Rated')

#Fill null values with Anonymous in Director column
netflix_data['director'] = netflix_data['director'].fillna('Anonymous')

#Fill null values with Unknown in Cast column
netflix_data['cast'] = netflix_data['cast'].fillna('Unknown')

#Fill null values with Unknown in Country column
netflix_data['country'] =netflix_data['country'].fillna('Unknown')


# netflix_data.info()

In [8]:
#Convert date added datatype into int
netflix_data['date_added'] = pd.to_datetime(netflix_data['date_added'], errors='coerce')

#Fill date_added column with Date: 01-01 of release year column
netflix_data['date_added'] = netflix_data['date_added'].fillna(pd.to_datetime(netflix_data['release_year'].astype(str) + '-01-01', errors='coerce'))

print(netflix_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       5837 non-null   int64         
 1   title         5837 non-null   object        
 2   director      5837 non-null   object        
 3   cast          5837 non-null   object        
 4   country       5837 non-null   object        
 5   date_added    5837 non-null   datetime64[ns]
 6   release_year  5837 non-null   int64         
 7   rating        5837 non-null   object        
 8   duration      5837 non-null   object        
 9   listed_in     5837 non-null   object        
 10  description   5837 non-null   object        
 11  type          5837 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 547.3+ KB
None


In [9]:
#Column Transformation

#Splitting date_added column into year and month
netflix_data['year_added'] = netflix_data['date_added'].dt.year 

netflix_data['month_added'] = netflix_data['date_added'].dt.month


In [10]:
import re

# Movies: extract minutes only
def extract_movie_minutes(duration):
    if pd.isna(duration):
        return 0
    # find the number before 'min'
    match = re.search(r'(\d+)\s*min', duration)
    if match:
        return float(match.group(1))
    return 0

# Shows: extract number of seasons only
def extract_show_seasons(duration):
    if pd.isna(duration):
        return 0
    # find number before 'Season' or 'Seasons'
    match = re.search(r'(\d+)\s*Season', duration)
    if match:
        return float(match.group(1))
    return 0

# Apply to dataframe
netflix_data['movie_duration'] = netflix_data.apply(
    lambda x: extract_movie_minutes(x['duration']) if x['type']=='Movie' else 0, axis=1
)

netflix_data['show_duration'] = netflix_data.apply(
    lambda x: extract_show_seasons(x['duration']) if x['type']=='TV Show' else 0, axis=1
)

# Check result
print(netflix_data[['title','type','duration','movie_duration','show_duration']].head(20))


                                     title     type   duration  \
0                                Chocolate  TV Show   1 Season   
1      Guatemala: Heart of the Mayan World    Movie     67 min   
2                          The Zoya Factor    Movie    135 min   
3                                Atlantics    Movie    106 min   
4                          Chip and Potato  TV Show  2 Seasons   
5                             Crazy people    Movie    107 min   
6                           I Lost My Body    Movie     81 min   
7   Kalushi: The Story of Solomon Mahlangu    Movie    107 min   
8                         La Reina del Sur  TV Show  2 Seasons   
9                     Lagos Real Fake Life    Movie    118 min   
10                                  Payday    Movie    110 min   
11                    Sugar Rush Christmas  TV Show   1 Season   
12                      The Accidental Spy    Movie    104 min   
13                    The Charming Stepmom  TV Show   1 Season   
14        

In [12]:
# Copy the original Dataframe for exploding
netflix_data_exploded = netflix_data.copy()

# Checking Unique Values in listed_in Column
print(netflix_data['listed_in'].unique())

# Split the listed_in Column into list
netflix_data_exploded['listed_in_split'] = netflix_data_exploded['listed_in'].str.split(',')

# Removes Spaces
netflix_data_exploded['listed_in_split'] = netflix_data_exploded['listed_in_split'].apply(lambda x: [i.strip() for i in x])

# Explode into multiple rows not columns
netflix_data_exploded = netflix_data_exploded.explode('listed_in_split')

print(netflix_data.info()) 


['International TV Shows, Korean TV Shows, Romantic TV Shows'
 'Documentaries, International Movies'
 'Comedies, Dramas, International Movies'
 'Dramas, Independent Movies, International Movies' "Kids' TV"
 'Comedies, International Movies, Thrillers'
 'Dramas, International Movies'
 'Crime TV Shows, International TV Shows, Spanish-Language TV Shows'
 'Comedies, International Movies'
 'Comedies, Independent Movies, International Movies' 'Reality TV'
 'Action & Adventure, Comedies, International Movies'
 'International TV Shows, Romantic TV Shows, TV Comedies'
 'Dramas, International Movies, Thrillers' 'Docuseries'
 'Children & Family Movies, Dramas' 'Anime Series, International TV Shows'
 'International TV Shows, Stand-Up Comedy & Talk Shows, TV Comedies'
 'TV Comedies' 'International TV Shows, TV Comedies, TV Dramas'
 'Dramas, Thrillers' 'Action & Adventure, Dramas'
 'Comedies, International Movies, Romantic Movies'
 'Children & Family Movies, Comedies' 'Dramas, Independent Movies'
 'D

In [13]:
# Checking Unique Values in cast Column
print(netflix_data['cast'].unique())

# Split the cast Column into list
netflix_data_exploded['cast_split'] = netflix_data_exploded['cast'].str.split(',')

# Removes Spaces
netflix_data_exploded['cast_split'] = netflix_data_exploded['cast_split'].apply(lambda x: [i.strip() for i in x])

# Explode into multiple rows not columns
netflix_data_exploded = netflix_data_exploded.explode('cast_split')



['Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang Bu-ja, Lee Jae-ryong, Min Jin-woong, Kim Won-hae, Yoo Teo'
 'Christian Morales'
 'Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, Sikander Kher, Angad Bedi, Koel Purie, Pooja Bhamrah, Manu Rishi Chadha'
 ...
 'Corey Feldman, Tony Todd, Tara Leigh, Erin Way, Mark Alan, Stuart Pankin'
 'Anders W. Berthelsen, Rebecka Hemse, Nikolaj Lie Kaas, Charlotte Fich, Dejan Cukic, Karsten Jansfort, Flemming Enevold, Bent Mejding, Ewa Fröling, Josephine Raahauge, Timm Vladimir, Ditte Hansen'
 'Barbara King, Shaana Diya, John Krisiukenas, Yoriko Haraguchi, David Crane, Chad Thomas']


In [14]:
# insights

# Movies vs TV shows Counts
types = netflix_data['type'].value_counts()
print(types)


type
Movie      3939
TV Show    1898
Name: count, dtype: int64


In [12]:
# Trend Over Years
trend = netflix_data.groupby(['year_added', 'type'])['show_id'].count()
print(trend)

year_added  type   
1963        TV Show       1
1967        TV Show       1
1968        TV Show       1
1974        TV Show       1
1977        TV Show       1
1986        TV Show       1
1988        TV Show       1
1990        TV Show       2
1992        TV Show       2
1993        TV Show       1
1995        TV Show       1
1997        TV Show       2
1998        TV Show       2
1999        TV Show       1
2000        TV Show       1
2001        TV Show       1
2003        TV Show       3
2004        TV Show       2
2005        TV Show       4
2006        TV Show       3
2007        TV Show       3
2008        Movie         1
            TV Show       8
2009        Movie         2
            TV Show       3
2010        Movie         1
            TV Show       9
2011        Movie        13
            TV Show       5
2012        Movie         4
            TV Show      18
2013        Movie         6
            TV Show      20
2014        Movie        19
            TV Show      18


In [13]:
# Content rating Distribution
Content_rating = netflix_data.groupby('rating')['show_id'].count().sort_values(ascending=False)
print(Content_rating)

rating
TV-MA        1937
TV-14        1593
TV-PG         678
R             439
PG-13         227
NR            218
PG            160
TV-Y7         156
TV-G          147
TV-Y          139
TV-Y7-FV       92
G              32
Not Rated      10
UR              7
NC-17           2
Name: show_id, dtype: int64


In [14]:
# top 10 Countries 
top_countries = netflix_data.groupby('country')['show_id'].count().sort_values(ascending=False)
print(top_countries.head(10))

country
United States     1907
India              697
Unknown            427
United Kingdom     336
Japan              168
Canada             139
South Korea        133
Spain              113
France              85
Mexico              80
Name: show_id, dtype: int64


In [15]:
# Top 10 Directors 
top_directors = netflix_data.groupby('director')['show_id'].count().sort_values(ascending=False)
print(top_directors.head(10))

director
Anonymous                 1901
Raúl Campos, Jan Suter      18
Marcus Raboy                14
Jay Karas                   13
Jay Chapman                 12
Martin Scorsese              8
Johnnie To                   8
Steven Spielberg             8
Cathy Garcia-Molina          7
Lance Bangs                  7
Name: show_id, dtype: int64


In [16]:
# Top 10 Actors
top_actors = netflix_data_exploded.groupby('cast_split')['show_id'].count().sort_values(ascending=False)
print(top_actors.head(10))

cast_split
Unknown             997
Anupam Kher          83
Shah Rukh Khan       80
Akshay Kumar         65
Yuki Kaji            57
Paresh Rawal         53
Om Puri              53
Takahiro Sakurai     53
Boman Irani          49
Naseeruddin Shah     48
Name: show_id, dtype: int64


In [17]:
# top 10 genres
genres = netflix_data_exploded.groupby('listed_in_split')['show_id'].count().sort_values(ascending=False)
print(genres)

listed_in_split
International Movies            13081
Dramas                          12427
Comedies                         8880
International TV Shows           7768
TV Dramas                        5413
Action & Adventure               4861
Independent Movies               4297
TV Comedies                      3218
Crime TV Shows                   3150
Children & Family Movies         3039
Thrillers                        2886
Romantic Movies                  2746
Kids' TV                         2360
Romantic TV Shows                2039
Horror Movies                    1911
Sci-Fi & Fantasy                 1544
Music & Musicals                 1411
Anime Series                     1364
TV Action & Adventure            1300
Spanish-Language TV Shows        1234
British TV Shows                 1167
Documentaries                    1090
Korean TV Shows                   825
Sports Movies                     789
TV Mysteries                      723
TV Sci-Fi & Fantasy               

In [18]:
# top 10 Actors per genre
actor_genre = netflix_data_exploded.groupby(['listed_in_split', 'cast_split'])['show_id'].count().sort_values(ascending=False).reset_index()
actor_genre_no_unknown = actor_genre[actor_genre['cast_split'] != 'Unknown']
print(actor_genre_no_unknown.head(10))


         listed_in_split      cast_split  show_id
9   International Movies     Anupam Kher       26
11  International Movies  Shah Rukh Khan       25
12                Dramas  Shah Rukh Khan       23
13  International Movies    Akshay Kumar       22
16  International Movies    Paresh Rawal       18
17  International Movies         Om Puri       18
18                Dramas     Anupam Kher       18
19  International Movies  Kareena Kapoor       17
21  International Movies     Boman Irani       15
22  International Movies  Gulshan Grover       15


In [19]:
# Seasonal Trend
seasonal_trend = netflix_data.groupby('month_added')['show_id'].count().sort_values(ascending=False)
print(seasonal_trend.head(5))

month_added
1     1031
10     580
11     544
3      504
8      454
Name: show_id, dtype: int64


In [20]:
valid_movies = netflix_data[netflix_data['movie_duration'] > 0]
longest_movie = valid_movies.loc[valid_movies['movie_duration'].idxmax()]
shortest_movie = valid_movies.loc[valid_movies['movie_duration'].idxmin()]

print("Longest Movie:", longest_movie['title'], f"({longest_movie['movie_duration']} min)")
print("Shortest Movie:", shortest_movie['title'], f"({shortest_movie['movie_duration']} min)")

# Longest & Shortest Show (ignore 0)
valid_shows = netflix_data[netflix_data['show_duration'] > 0]
longest_show = valid_shows.loc[valid_shows['show_duration'].idxmax()]
shortest_show = valid_shows.loc[valid_shows['show_duration'].idxmin()]

print("Longest Show:", longest_show['title'], f"({longest_show['show_duration']} Seasons)")
print("Shortest Show:", shortest_show['title'], f"({shortest_show['show_duration']} Season)")




Longest Movie: Black Mirror: Bandersnatch (312.0 min)
Shortest Movie: Silent (3.0 min)
Longest Show: Grey's Anatomy (15.0 Seasons)
Shortest Show: Chocolate (1.0 Season)


In [21]:
netflix_data.to_excel("netflix_clean.xlsx", index=False)
